## Solvers' Testing

In [1]:
# Add locations of modules
push!(LOAD_PATH, "../src/")

3-element Array{Any,1}:
 "/home/local/julia/julia-d386e40c17/local/share/julia/site/v0.6"
 "/home/local/julia/julia-d386e40c17/share/julia/site/v0.6"      
 "../src/"                                                       

## Stochastic Gradient Descent

## Test 1

In [2]:
using GraphicalModelLearning

srand(0)

# Sample considering a simple factor graph
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])
samples1 = sample(model, 5000)

# Sample using the Gibbs Sampler
n_samples = 100000
samples_time_series = gibbs_sampling(model, n_samples)

# Every mth sample after burn in (Determining when burn-in happens?)
t_burn_in = 50000
m_samples = 10

samples_iid = samples_time_series[t_burn_in:m_samples:n_samples,:]

# Compare samples against those produced by direct method
using StatsBase
raw_binning = countmap(samples_iid)
samples2 = [ vcat(raw_binning[i], 2*digits(i, 2, model.varible_count)-1) for i in keys(raw_binning)]
samples2 = hcat(samples2...)'

# Other trends to check samples after a long time are decorrelated and belong to the stationary distribution

INFO: Recompiling stale cache file /home/local/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.
INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate samples


8×4 Array{Int64,2}:
 1309  -1  -1  -1
  820  -1  -1   1
 1306   1   1   1
  182  -1   1  -1
  848   1   1  -1
  180   1  -1   1
  180  -1   1   1
  176   1  -1  -1

In [3]:
samples2[:,1]/sum(samples2[:,1])

8-element Array{Float64,1}:
 0.261748 
 0.163967 
 0.261148 
 0.0363927
 0.169566 
 0.0359928
 0.0359928
 0.035193 

In [4]:
samples1[:,1]/sum(samples1[:,1])

8-element Array{Float64,1}:
 0.2614
 0.1668
 0.2532
 0.0362
 0.1786
 0.0354
 0.0348
 0.0336

### Learning on iid samples

In [5]:
# Can the FactorGraph be recovered?
@time learned1 = learn(samples1)
@time learned2 = learn(samples2)

@time learned_sgd_rise1 = learn_sgd(samples1)
@time learned_sgd_rise2 = learn_sgd(samples2)

@time learned_sgd_rple1 = learn_sgd(samples1, RPLE())
@time learned_sgd_rple2 = learn_sgd(samples2, RPLE())

println(learned1)
println(learned2)

println(learned_sgd_rise1)
println(learned_sgd_rise2)

println(learned_sgd_rple1)
println(learned_sgd_rple2)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  8.007609 seconds (5.18 M allocations: 260.489 MiB, 1.36% gc time)
  0.019208 seconds (5.27 k allocations: 363.656 KiB)


INFO: using stochastic gradient solver for RISE


  8.188973 seconds (241.78 M allocations: 4.701 GiB, 6.96% gc time)


INFO: using stochastic gradient solver for RISE


  7.710961 seconds (241.96 M allocations: 4.700 GiB, 7.13% gc time)


INFO: using stochastic gradient solver for RPLE


  7.602876 seconds (231.20 M allocations: 4.416 GiB, 6.86% gc time)


INFO: using stochastic gradient solver for RPLE


  3.196218 seconds (100.02 M allocations: 1.909 GiB, 7.08% gc time)
[-0.00385463 0.880962 0.0986432; 0.880962 0.00991948 0.0762026; 0.0986432 0.0762026 -0.0209155]
[-0.000826041 0.863831 0.104378; 0.863831 0.00735092 0.0961333; 0.104378 0.0961333 -0.0071306]
[-0.000286178 0.87706 0.101103; 0.87706 0.00230491 0.0761805; 0.101103 0.0761805 -0.00869534]
[0.000213814 0.861309 0.107809; 0.861309 0.000511856 0.0961263; 0.107809 0.0961263 -0.00120892]
[-0.00423124 0.886045 0.105687; 0.886045 0.000581257 0.0769122; 0.105687 0.0769122 -0.0153581]
[-0.0034631 0.862953 0.109057; 0.862953 -0.000414161 0.0969086; 0.109057 0.0969086 -3.84933e-5]


### Learning on correlated samples -- T-regime

In [9]:
using GraphicalModelLearning

srand(0)

# Define Graphical Model
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])

n_samples = 5000
# Plain sampling
samples1 = sample(model, n_samples)

# Gibbs Samples
samples_T, samples_mixed = gibbs_sampling(model, n_samples, T_regime())
#gibbs_output = gibbs_sampling(model, n_samples, T_regime())
#samples_T = gibbs_output[1]
#samples_mixed = gibbs_output[2]
print("Done sampling")

INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate T-regime samples


Done sampling

In [10]:
samples_T

32×7 Array{Int64,2}:
   50   1  -1  -1  -1  -1  -1
   55   1   1   1   1  -1   1
   47  -1   1   1  -1   1   1
  175  -1  -1  -1  -1  -1   1
   27  -1   1  -1  -1   1   1
   52  -1   1  -1  -1  -1  -1
   46  -1  -1  -1  -1   1  -1
   55  -1   1   1   1   1   1
   45   1   1  -1   1  -1  -1
 1038  -1  -1  -1  -1  -1  -1
   67   1  -1  -1   1  -1  -1
   60   1  -1  -1   1   1  -1
   39  -1   1  -1   1   1  -1
    ⋮                   ⋮    
   54   1  -1   1  -1  -1   1
   48   1  -1   1   1   1   1
   47  -1   1   1  -1  -1   1
   57   1  -1   1   1  -1   1
  929   1   1   1   1   1   1
   30   1  -1   1   1  -1  -1
  175   1   1   1   1   1  -1
  614   1   1  -1   1   1  -1
  566  -1  -1   1  -1  -1   1
   29   1  -1  -1   1  -1   1
   45  -1  -1   1  -1   1   1
   46  -1   1  -1  -1   1  -1

In [7]:
samples_mixed

8×4 Array{Int64,2}:
 1323  -1  -1  -1
  842  -1  -1   1
 1214   1   1   1
  164  -1   1  -1
  888   1   1  -1
  189   1  -1   1
  174  -1   1   1
  206   1  -1  -1

In [8]:
@time learned3 = learn_glauber_dynamics(samples_T)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  8.807619 seconds (5.15 M allocations: 259.367 MiB, 1.07% gc time)


3×3 Array{Float64,2}:
 0.031891    0.85722     0.0618882
 0.85722    -0.0458952   0.0975106
 0.0618882   0.0975106  -0.0304779

In [10]:
@time learned4 = learn_glauber_dynamics_sgd(samples_T)

INFO: using stochastic gradient solver for RISE


  0.998540 seconds (12.40 M allocations: 232.713 MiB, 4.54% gc time)


3×3 Array{Float64,2}:
 -0.075218    0.820355    0.0707643
  0.820355   -0.0373161   0.126903 
  0.0707643   0.126903   -0.0495768

In [11]:
@time learned5 = learn_glauber_dynamics_sgd(samples_T, RISE(), 8.0, 1e-10)

INFO: using stochastic gradient solver for RISE


  6.074771 seconds (118.32 M allocations: 2.119 GiB, 4.99% gc time)


3×3 Array{Float64,2}:
 -0.0601043   0.85288     0.0992113
  0.85288    -0.0639196   0.106892 
  0.0992113   0.106892   -0.0366838

In [12]:
@time learned5 = learn_glauber_dynamics_sgd(samples_T, RISE(), 8.0, 1e-11)

INFO: using stochastic gradient solver for RISE


 14.826829 seconds (287.32 M allocations: 5.145 GiB, 4.95% gc time)


3×3 Array{Float64,2}:
 -0.0612704   0.860571    0.0952477
  0.860571   -0.0421665   0.105183 
  0.0952477   0.105183   -0.0320085

In [13]:
@time learned5 = learn_glauber_dynamics_sgd(samples_T, RPLE())

INFO: using stochastic gradient solver for RPLE


  1.318468 seconds (19.04 M allocations: 349.709 MiB, 4.17% gc time)


3×3 Array{Float64,2}:
 -0.0443964   0.848512    0.101827 
  0.848512   -0.0369464   0.108831 
  0.101827    0.108831   -0.0518408

In [14]:
@time learned5 = learn_glauber_dynamics_sgd(samples_T, RPLE(), 8.0, 1e-10)

INFO: using stochastic gradient solver for RPLE


  6.092684 seconds (111.45 M allocations: 1.969 GiB, 4.75% gc time)


3×3 Array{Float64,2}:
 -0.0456916   0.868695    0.0953161
  0.868695   -0.0434767   0.103668 
  0.0953161   0.103668   -0.0405221

### Learning on correlated samples -- M-regime

In [2]:
using GraphicalModelLearning

srand(0)

# Define Graphical Model
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#model = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])

n_samples = 5000
# Plain sampling
samples1 = sample(model, n_samples)

# Gibbs Samples
samples_M, samples_mixed = gibbs_sampling(model, n_samples, M_regime())
#gibbs_output = gibbs_sampling(model, n_samples, T_regime())
#samples_T = gibbs_output[1]
#samples_mixed = gibbs_output[2]
print("Done sampling")

INFO: Recompiling stale cache file /home/local/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.
INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate M-regime samples


Done sampling

In [3]:
samples_M

32×7 Array{Int64,2}:
 199   1  -1  -1  -1  -1  -1
  23   1   1   1   1  -1   1
 175  -1   1   1  -1   1   1
  69  -1  -1  -1  -1  -1   1
  93  -1   1  -1  -1   1   1
 168  -1   1  -1  -1  -1  -1
  23  -1  -1  -1  -1   1  -1
 179  -1   1   1   1   1   1
  47   1   1  -1   1  -1  -1
 501  -1  -1  -1  -1  -1  -1
 168   1  -1  -1   1  -1  -1
 182   1  -1  -1   1   1  -1
 168  -1   1  -1   1   1  -1
   ⋮                   ⋮    
 178   1  -1   1  -1  -1   1
 160   1  -1   1   1   1   1
 161  -1   1   1  -1  -1   1
 176   1  -1   1   1  -1   1
 513   1   1   1   1   1   1
 114   1  -1   1   1  -1  -1
  88   1   1   1   1   1  -1
 417   1   1  -1   1   1  -1
 429  -1  -1   1  -1  -1   1
  91   1  -1  -1   1  -1   1
  36  -1  -1   1  -1   1   1
 166  -1   1  -1  -1   1  -1

In [8]:
samples_mixed

8×4 Array{Int64,2}:
 1006  -1  -1  -1
  837  -1  -1   1
  980   1   1   1
  320  -1   1  -1
  855   1   1  -1
  329   1  -1   1
  322  -1   1   1
  351   1  -1  -1

In [5]:
@time learned1 = learn_glauber_dynamics(samples_M)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

  8.998982 seconds (5.16 M allocations: 259.507 MiB, 0.95% gc time)


3×3 Array{Float64,2}:
 0.00892352   0.232999    0.0100643
 0.232999    -0.0132189   0.0210828
 0.0100643    0.0210828  -0.0126277

In [6]:
@time learned2 = learn(samples_mixed)

  0.265190 seconds (70.67 k allocations: 3.558 MiB)


3×3 Array{Float64,2}:
 0.0132895   0.496226    0.0197846
 0.496226   -0.0148107   0.0374478
 0.0197846   0.0374478  -0.012612 